In [1]:
import pathlib, warnings
warnings.filterwarnings('ignore')

import pandas as pd
import plotly.express as px
print(pd.__version__)


DATA_DIR = pathlib.Path('data')

2.2.1


In [52]:
IN_COLUMNS = [
    'in_spotify_playlists', 'in_spotify_charts',
    'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists',
    'in_deezer_charts', 'in_shazam_charts'
]
songs = (pd
    .read_csv(DATA_DIR / 'spotify-2023.csv', encoding='latin-1')
    .rename(columns={'released_day': 'day', 'released_month': 'month', 'released_year': 'year'})
    .astype({'key': 'category', 'mode': 'category'})
)
songs.insert(3, 'released_date', pd.to_datetime(songs[['year', 'month', 'day']]))
songs_charts = songs[IN_COLUMNS]
songs = songs.drop(columns=['day', 'month', 'year'] + IN_COLUMNS)
songs.columns = [col.replace('(', '').replace(')', '').replace('_%', '') for col in songs.columns]
songs['streams'] = pd.to_numeric(songs['streams'], errors='coerce').fillna(0).astype(int)

In [53]:
songs.columns

Index(['track_name', 'artists_name', 'artist_count', 'released_date',
       'streams', 'bpm', 'key', 'mode', 'danceability', 'valence', 'energy',
       'acousticness', 'instrumentalness', 'liveness', 'speechiness'],
      dtype='object')

In [54]:
songs.head()

,track_name,artists_name,artist_count,released_date,streams,bpm,key,mode,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023-07-14,141381703,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023-03-23,133716286,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023-06-30,140003974,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019-08-23,800840817,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023-05-18,303236322,144,A,Minor,65,23,80,14,63,11,6


In [5]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   track_name        953 non-null    object        
 1   artists_name      953 non-null    object        
 2   artist_count      953 non-null    int64         
 3   released_date     953 non-null    datetime64[ns]
 4   streams           953 non-null    int32         
 5   bpm               953 non-null    int64         
 6   key               858 non-null    category      
 7   mode              953 non-null    category      
 8   danceability      953 non-null    int64         
 9   valence           953 non-null    int64         
 10  energy            953 non-null    int64         
 11  acousticness      953 non-null    int64         
 12  instrumentalness  953 non-null    int64         
 13  liveness          953 non-null    int64         
 14  speechiness       953 non-

In [6]:
songs.describe()

,artist_count,released_date,streams,bpm,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness
count,953.000000,953,9.530000e+02,953.000000,953.00000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000
mean,1.556139,2018-09-10 05:09:45.519412480,3.899776e+08,122.540399,66.96957,51.431270,64.279119,27.057712,1.581322,18.213012,10.131165
min,1.000000,1930-01-01 00:00:00,-2.147484e+09,65.000000,23.00000,4.000000,9.000000,0.000000,0.000000,3.000000,2.000000
25%,1.000000,2020-06-28 00:00:00,1.306558e+08,100.000000,57.00000,32.000000,53.000000,6.000000,0.000000,10.000000,4.000000
50%,1.000000,2022-04-08 00:00:00,2.677585e+08,121.000000,69.00000,51.000000,66.000000,18.000000,0.000000,12.000000,6.000000
75%,2.000000,2022-11-04 00:00:00,6.071238e+08,140.000000,78.00000,70.000000,77.000000,43.000000,0.000000,24.000000,11.000000
max,8.000000,2023-07-14 00:00:00,2.135158e+09,206.000000,96.00000,97.000000,97.000000,97.000000,91.000000,97.000000,64.000000
std,0.893044,NaN,6.145664e+08,28.057802,14.63061,23.480632,16.550526,25.996077,8.409800,13.711223,9.912888


# Analysis

In [34]:
CHARACTERISTICS = [
    'danceability', 'valence',
    'energy', 'acousticness',
    'instrumentalness', 'liveness',
    'speechiness',
]

In [35]:
characteristics_by_key_bars = px.bar(
    (songs
        .groupby('key')
        [CHARACTERISTICS]
        .mean()
    ), 
    barmode='group',
    title='Characteristics by Key',
    labels={'value': 'Mean Value', 'key': 'Key', 'variable': 'Characteristic'},
    template='plotly_dark'
)
characteristics_by_key_bars.update_layout(legend=dict(orientation='h',yanchor='top',y=1.15,xanchor='right',x=1))
characteristics_by_key_bars.show()

In [40]:
characteristics_by_mode_bars = px.bar(
    (songs
        .groupby(['mode'])
        [CHARACTERISTICS]
        .mean()
        .transpose()
    ), 
    barmode='group',
    title='Characteristics by Mode',
    labels={'value': 'Mean Value', 'index': 'Characteristic'},
    template='plotly_dark'
)
characteristics_by_mode_bars.update_layout(legend=dict(orientation='h',yanchor='top',y=1.15,xanchor='right',x=1))
characteristics_by_mode_bars.show()

In [50]:
streams_bpm_by_year = (
    songs
    .groupby(songs.released_date.dt.year)
    [['streams', 'bpm']]
    .mean()
)

In [56]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=streams_bpm_by_year.index, y=streams_bpm_by_year['streams'], name="Streams"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=streams_bpm_by_year.index, y=streams_bpm_by_year['bpm'], name="BPM"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Double Y Axis Example",
    template='plotly_dark'
)

# Set x-axis title
fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

fig.show()

In [60]:
(
    songs
    .join(songs_charts)
    .groupby(songs.released_date.dt.year)
    [IN_COLUMNS]
    .mean()
)

TypeError: agg function failed [how->mean,dtype->object]

In [66]:
songs_charts.in_deezer_playlists.str.replace(',', '').astype(int)

0       45
1       58
2       91
3      125
4       87
      ... 
948     37
949      8
950      7
951     17
952     32
Name: in_deezer_playlists, Length: 953, dtype: int32

In [75]:
songs_charts.in_shazam_charts.str.replace(',', '').astype(int, errors='coerce')

ValueError: Expected value of kwarg 'errors' to be one of ['raise', 'ignore']. Supplied value is 'coerce'